In [33]:
!pip install pandas
!pip install html5lib

     |████████████████████████████████| 117 kB 1.7 MB/s eta 0:00:01


In [8]:
!python3 -m spacy download es_core_news_sm


✘ No compatible model found for 'es_core_web_sm' (spaCy v2.2.4).



In [9]:
!python3 -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [56]:
!pip install contractions inflect

In [98]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

import html5lib
import lxml
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import spacy

[nltk_data] Downloading package punkt to /home/carlosb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/carlosb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/carlosb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [111]:
class cleantext():
    
    def __init__(self, text = "test"):
        self.text = text
        
    def strip_html(self):
        soup = BeautifulSoup(self.text, "html.parser")
        self.text = soup.get_text()
        return self

    def remove_between_square_brackets(self):
        self.text = re.sub('\[[^]]*\]', '', self.text)
        return self

    def remove_numbers(self):
        self.text = re.sub('[-+]?[0-9]+', '', self.text)
        return self

    def replace_contractions(self):
        """Replace contractions in string of text"""
        self.text = contractions.fix(self.text)
        return self
    
    def get_words(self):
        self.words = nltk.word_tokenize(self.text)
        return self

    def remove_non_ascii(self):
        """Remove non-ASCII characters from list of tokenized words"""
        new_words = []
        for word in self.words:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        self.words = new_words
        return self

    def to_lowercase(self):
        """Convert all characters to lowercase from list of tokenized words"""
        new_words = []
        for word in self.words:
            new_word = word.lower()
            new_words.append(new_word)
        self.words = new_words
        return self

    def remove_punctuation(self):
        """Remove punctuation from list of tokenized words"""
        new_words = []
        for word in self.words:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
        self.words = new_words
        return self

    def replace_numbers(self):
        """Replace all interger occurrences in list of tokenized words with textual representation"""
        p = inflect.engine()
        new_words = []
        for word in self.words:
            if word.isdigit():
                new_word = p.number_to_words(word)
                new_words.append(new_word)
            else:
                new_words.append(word)
        self.words = new_words
        return self

    def remove_stopwords(self):
        """Remove stop words from list of tokenized words"""
        new_words = []
        for word in self.words:
            if word not in stopwords.words('english'):
                new_words.append(word)
        self.words = new_words
        return self

    def stem_words(self):
        """Stem words in list of tokenized words"""
        stemmer = LancasterStemmer()
        stems = []
        for word in self.words:
            stem = stemmer.stem(word)
            stems.append(stem)
        self.words = stems
        return self

    def lemmatize_verbs(self):
        """Lemmatize verbs in list of tokenized words"""
        lemmatizer = WordNetLemmatizer()
        lemmas = []
        for word in self.words:
            lemma = lemmatizer.lemmatize(word, pos='v')
            lemmas.append(lemma)
        self.words = lemmas
        return self
    
    def join_words(self):
        self.words = ' '.join(self.words)
        return self
    def spacy_tokenize(self):
        nlp = spacy.load('en_core_web_sm')
        nlp.remove_pipe('parser')
        nlp.remove_pipe('ner')
        nlp.remove_pipe('tagger')
        # Replace all newlines and blanklines with special strings
        regx = re.compile(r"\n")
        self.text = regx.sub(repl=" ", string=self.text)
        regx = re.compile(r"\n\n")
        self.text = regx.sub(repl=" ", string=self.text)

        # Make all white space a single space
        regx = re.compile(r"\s+")
        self.text = regx.sub(repl=" ", string=self.text)

        # Remove any trailing or leading white space
        self.text = self.text.strip(" ")
        
        self.words = [token.lower_ for token in  list(nlp.pipe([self.text], n_threads=1))[0]]
        return self
    
    def do_all(self, text,spacy=False):
        
        self.text = text
        self = self.strip_html()
        self = self.remove_numbers()
        self = self.replace_contractions()
        if spacy:
            self = self.spacy_tokenize()
        else:
            self = self.get_words()
        self = self.remove_punctuation()
        self = self.remove_non_ascii()
        self = self.remove_stopwords()
        self = self.stem_words()
        self = self.lemmatize_verbs()
        self = self.join_words()
        
        return self.words

In [112]:
f = open('./test_html_article')
html = f.read()

In [113]:
cleantext().do_all(html,spacy=True)

'prepar est irresist sandwich cubanito es as de facil hoy inter carn cer fruta fruto seco legumbr pescado tuberculo verdura mas secc al minuto internac politic vid deport econom opin gent cultur suceso participac blog rcd espanyol europ fact check suscrib tema can la contr nat big vang salud tecnolog televid sery muy fan cribeo magazin lifestyl vias vivo mot de mod com vivo seguro can andorr el comprad histor vid ed loc madrid barcelon cataluny andaluc comunidad valencian pai vasco ver mas suplemento cultura vanguard dossy vanguard grand tema libro de vanguard clasificado monografico can relod servicio edicio en catal ed impres hemerotec bols programac tv loteria horoscopo juego carteler agend juny report rss necrologica esquela gourmet la vanguard lv shop entrada de vanguard pista de esqu header_logo_text al dia opin sitio tendencia mater prim receta de carn hueso video receta prepar est irresist sandwich cubanito es as de facil un plato sencillo pero muy apetitoso prepar est apetitos

('tagger', <spacy.pipeline.pipes.Tagger at 0x7fb9304db460>)

'hoy interesa carnes cereales frutas frutos secos legumbres pescados tubérculos verduras mas secciones al minuto internacional política vida deportes economía opinión gente cultura sucesos participación blogs rcd espanyol europa fact checking suscriptores temas canales la contra natural big vang salud tecnología televisión series muy fan cribeo magazine lifestyle viajes vivo motor de moda comer vivo seguro canal andorra el comprador historia y vida ediciones locales madrid barcelona catalunya andalucía comunidad valenciana país vasco ver más suplementos cultura|s vanguardia dossier vanguardia grandes temas libros de vanguardia clasificados monográficos canal relojes servicios edició en català edición impresa hemeroteca bolsa programación tv loterías horóscopo juegos cartelera agenda junior report rss necrológicas y esquelas gourmet la vanguardia lv shopping entradas de vanguardia pistas de esquí header_logo_text al día opinión sitios tendencias materia prima recetas de carne y hueso vi

In [19]:
import spacy
nlp = spacy.load("es_core_news_sm")

In [20]:
import gensim.downloader as api

In [21]:
model = api.load("glove-twitter-25")

[==================================================] 100.0% 104.8/104.8MB downloaded


In [24]:
from word2number import w2n
doc = nlp(visible_text)
tokens = [w2n.word_to_num(token.text) if token.pos_ == 'NUM' else token for token in doc]

print(tokens) # result: [3, cups, of, coffee]

ValueError: No valid number words found! Please enter a valid number word (eg. two million twenty three thousand and forty nine)

In [25]:

doc = nlp(visible_text)
# Lemmatizing each token
mytokens = [word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in doc]
print(mytokens) 
# result: ['he', 'keep', 'eat', 'while', 'we', 'be', 'talk']

[' \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Hoy', 'interesar', '\n\n', 'Carnes', '\n\n\n', 'Cereales', '\n\n\n', 'Frutas', '\n\n\n', 'Frutos', 'seco', '\n\n\n', 'Legumbres', '\n\n\n', 'Pescados', '\n\n\n', 'Tubérculos', '\n\n\n', 'Verduras', '\n\n\n\n\n\n', 'Mas', '\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'Secciones', '\n\n\n\n', 'Al', 'Minuto', '\n\n\n', 'Internacional', '\n\n\n', 'Política', '\n\n\n', 'Vida', '\n\n\n', 'Deportes', '\n\n\n', 'Economía', '\n\n\n', 'Opinión', '\n\n\n', 'Gente', '\n\n\n', 'Cultura', '\n\n\n', 'Sucesos', '\n\n\n', 'Participación', '\n\n\n', 'Blogs', '\n\n\n', 'RCD', 'Espanyol', '\n\n\n', 'Europa', 'Fact', 'Checking', '\n\n\n', 'Suscriptores', '\n\n\n', 'Temas', '\n\n\n\n\n\n', 'Canales', '\n\n\n\n', 'La', 'Contra', '\n\n\n', 'Natural', '\n\n\n', 'Big', 'Vang', '\n\n\n', 'Salud', '\n\n\n', 'Tecnología', '\n\n\n', 'Televisión', '\n\n\n', 'Series', '\n\n\n', 'Muy', 'Fan', '\n\n\n', 'Cribeo', '\n\n\n', 'Magazine', 'Lifestyle', '\n